In [42]:
import random
from dataclasses import dataclass, field
from typing import List, Dict, Any

import numpy as np
import os
from tampura.environment import TampuraEnv
from tampura.spec import ProblemSpec
from tampura.structs import (
    AbstractBelief,
    ActionSchema,
    StreamSchema,
    AliasStore,
    Belief,
    NoOp,
    Predicate,
    State,
    effect_from_execute_fn,
    Observation
)
import logging 
from tampura.symbolic import OBJ, Or, Atom, ForAll, Exists, OneOf, And, Not, Eq, negate
from tampura.policies.tampura_policy import TampuraPolicy
from tampura.config.config import get_default_config, setup_logger


PICK_SUCCESS = 1.0
DROP_SUCCESS = 1.0

REGIONS={"r1":[0.0,0.0,0.0],"r2":[0.0,1.0,0.0],"r3":[1.0,0.0,0.0],"r4":[1.0,1.0,0.0],"robot_gripper":[0.5,0.5,0.5],"pile":[2.0,0.0,0.0]}
CONFIGURATIONS={"q1":[0.0,0.0,0.0],"q2":[0.0,1.0,0.0],"q3":[1.0,0.0,0.0],"q4":[1.0,1.0,0.0]}

OBJECTS = {f"{OBJ}o1":{"region":"r1","size":10},f"{OBJ}o2":{"region":"r2","size":4}}
ROBOT = {"configuration":"q1"}
DICE = {"region": "r2","size":1}

THRESH = 0.5

In [ ]:
# Observation space


@dataclass 
class ObjectObservation(Observation):
    
    at: Dict[str,str]=field(default_factory=lambda: {})
    atRob: str=field(default_factory=lambda: "")
    looked: List[str]=field(default_factory=lambda: [])
    

# Belief space
class ObjectsBelief(Belief):
    def __init__(self, at={}, atRob="", looked=[]):
        
        self.at=at
        self.atRob=atRob
        self.looked=looked

  

    def update(self, a, o, s):
        
        at=o.at
        atRob=o.atRob
        looked=o.looked
        
        return ObjectsBelief(at=at,atRob=atRob,looked=looked) 

    def abstract(self, store: AliasStore):
        
        atoms = []
        
        if self.atRob != "":
            atoms.append(Atom("atRob",[self.atRob]))
        
        for o in self.at.keys():
            atoms.append(Atom("at",[o,self.at[o]]))
        
        for q in self.looked:
            atoms.append(Atom("looked",[q]))
            
            
        return AbstractBelief(atoms)
    

    def vectorize(self):
        pass
    
# Sample function for stream schema
    
def sample_grasp(input_sym, store):
    
    g = 1.0
    
    # TODO: make more sophisticated
    
    return g

    
# action simulators
def isClose(loc1,loc2,thresh=0.5):
    
    return np.linalg.norm(np.array(loc1)-np.array(loc2))<thresh

def forward_kinematics(q,store):
    
    for k,v in store.als_type.items():
        if v=="region":
            if store.get(k) == store.get(q):
                return k
            
    return "None"
    
# Action simulators
def look_execute_fn(a, b, s, store):
    
    q = a.args[0]
    r = a.args[1]
    
    at=b.at.copy()
    atRob=b.atRob
    looked=b.looked.copy()
    
    size=0
    flag=False
    obj=None
    for o in OBJECTS.keys():
        if at.get(o) is None: # not already located, meaning it is at initial location
            
            if isClose(store.get(OBJECTS[o]["region"]),store.get(r)) and OBJECTS[o]["size"]>size:
                obj=o
                size=OBJECTS[o]["size"]
                
    
                
    if isClose(store.get(DICE["region"]),store.get(r)) and DICE["size"]>size:
                obj="dice"
                size=DICE["size"]
                
                
    if obj is None:
        looked = list(set(looked+[q]))
    else:
        at[obj]=r
    
            
    return State(), ObjectObservation(at=at,atRob=atRob,looked=looked)
    

    
def move_execute_fn(a, b, s, store):
    
    q = a.args[0]
    
    at=b.at.copy()
    atRob=b.atRob
    looked=b.looked.copy()
    
    # effects
    atRob=q
    
    return State(), ObjectObservation(at=at,atRob=atRob,looked=looked)
   
def pick_and_drop_execute_fn(a, b, s, store):
    
    o = a.args[0]
    g = a.args[1]
    
    at=b.at.copy()
    atRob=b.atRob
    looked=b.looked.copy()
    
    # verify_effects
    if random.random() < PICK_SUCCESS:
        at[o]="pile"
    
    return State(), ObjectObservation(at=at,atRob=atRob,looked=looked) 

def pick_execute_fn(a, b, s, store):
    
    o = a.args[0]
    g = a.args[1]
    
    at=b.at.copy()
    atRob=b.atRob
    looked=b.looked.copy()

    
    # verify_effects
    if random.random() < PICK_SUCCESS:
        at[o]="robot_gripper"
    
    return State(), ObjectObservation(at=at,atRob=atRob,looked=looked)


    
def drop_execute_fn(a, b, s, store):
    
    o = a.args[0]
    g = a.args[1]

    at=b.at.copy()
    atRob=b.atRob
    looked=b.looked.copy()
    
    # verify_effects
    if random.random() < DROP_SUCCESS:
        at[o]="pile"
        
    return State(), ObjectObservation(at=at,atRob=atRob,looked=looked)

    
# closed world assumption: not explicitly True predicates are false

# Set up environment dynamics
class ToyDiscrete(TampuraEnv):
    def initialize(self):
        
        store = AliasStore()
        
        # initialize store
        # set all object regions
        for r in REGIONS.keys():
            store.set(r, REGIONS[r], "region")
            
        # set all robot configurations
        for q in CONFIGURATIONS.keys():
            store.set(q, CONFIGURATIONS[q], "robot_configuration")
            r = forward_kinematics(q,store) # get corresponding region
            store.certified.append(Atom("points_to",[q,r])) # certified is OK for this, not fluent
            
        # object we are looking for
        store.set("dice", "dice", "physical")
        
        # set objects 
        for o in OBJECTS.keys():
            store.set(o, o, "physical")
        
        # initial belief
        
        
        # initial robot configuration        
        atRob=ROBOT["configuration"]
        
                   
        return ObjectsBelief(atRob=atRob), store
    

    def get_problem_spec(self) -> ProblemSpec:
        predicates = [ 
            Predicate("at",["physical","region"]),
            Predicate("atRob", ["robot_configuration"]),
            Predicate("grasped",["physical","grasp"]),
            Predicate("points_to", ["robot_configuration","region"]),
            Predicate("looked", ["robot_configuration"]),
        ] 
        
        stream_schemas = [
         
            StreamSchema(
                name="sample-grasp",
                inputs=["?o1"],
                input_types=["physical"],
                output="?g1",
                output_type="grasp",
                certified=[Atom("grasped",["?o1","?g1"])], # generates a valid grasp g1 for o1
                sample_fn=sample_grasp                
            ),
          
        ]
        
        action_schemas = [
            
            ActionSchema(
                name="look",
                inputs=["?q1","?r1"],
                input_types=["robot_configuration","region"],
                preconditions=[Not(Exists(Atom("at",["?o","robot_gripper"]),["?o"],["physical"])), # robot gripper is free
                               Atom("atRob",["?q1"]), # robot is at the configuration it is trying to inspect
                               Atom("points_to",["?q1","?r1"]), # ?q1 corresponds to ?r1
                               Not(Exists(Atom("at",["?o","?r1"]),["?o"],["physical"])), # no known object is in the region
                               Not(Atom("looked",["?q1"])), # has not been inspected already
                               ],
                
                verify_effects=[Atom("at",[o,"?r1"])for o in OBJECTS.keys()]+[Atom("at",["dice","?r1"]),Atom("looked",["?q1"])], 
                execute_fn=look_execute_fn,
                effects_fn=effect_from_execute_fn(look_execute_fn),
            
            ),
            
            
            ActionSchema(
                name="move-robot",
                inputs=["?q1"],
                input_types=["robot_configuration"],
                preconditions=[
                               Not(Exists(Atom("at",["?o","robot_gripper"]),["?o"],["physical"])), # robot gripper is free
                               Not(Atom("looked",["?q1"])), # do not move to a configuration that has already been inspected
                               Exists(Atom("atRob",["?q"]),["?q"],["robot_configuration"]), # current location of robot is known
                               Not(Atom("atRob",["?q1"])), # robot is not already at region it is moving to
                               ],
                effects=[Atom("atRob",["?q1"])], 
                execute_fn=move_execute_fn,
                effects_fn=effect_from_execute_fn(move_execute_fn),              
            
            ),
            

            ActionSchema(
                name="pick",
                inputs=["?o1","?g1","?r1"],
                input_types=["physical","grasp","region"],
                preconditions=[Not(Atom("at",["?o1","pile"])),
                               Atom("at",["?o1","?r1"]), # location of ?o1 is known
                               Atom("grasped",["?o1","?g1"]), # ?g1 is a valid grasp for ?o1
                               Not(Exists(Atom("at",["?o","robot_gripper"]),["?o"],["physical"])), # robot gripper is free
                               ], 
                verify_effects=[Atom("at",["?o1","robot_gripper"]),Not(Atom("at",["?o1","?r1"]))],
                execute_fn=pick_execute_fn,
                effects_fn=effect_from_execute_fn(pick_execute_fn),
            ),
            
            ActionSchema(
                name="drop",
                inputs=["?o1","?g1"],
                input_types=["physical","grasp"],
                preconditions=[
                               Atom("at",["?o1","robot_gripper"]), # robot is holding ?o1
                               Atom("grasped",["?o1","?g1"]), # ?g1 is a valid grasp for ?o1
                               Not(Atom("at",["?o1","pile"])), # current location is not the same as goal location
                               ], 
                verify_effects=[Atom("at",["?o1","pile"]),Not(Atom("at",["?o1","robot_gripper"]))], 
                execute_fn=drop_execute_fn,
                effects_fn=effect_from_execute_fn(drop_execute_fn),
            ),
            NoOp(),
        ]
        
        reward = Atom("at",["dice","robot_gripper"])
        
        spec = ProblemSpec(
            predicates=predicates,
            stream_schemas=stream_schemas,
            action_schemas=action_schemas,
            reward=reward,
        )

        return spec


## Create environment and planner

In [56]:
# Planner
cfg = get_default_config(save_dir=os.getcwd())

# Set some print options to print out abstract belief, action, observation, and reward
cfg["print_options"] = "ab,a,o,r"
cfg["vis_graph"] = True
cfg["flat_sample"] = True
cfg["batch_size"] = 100
cfg["num_samples"] = 1000
cfg["max_steps"] = 30


# Initialize environment
env = ToyDiscrete(config=cfg)
b0, store = env.initialize()

# Set up logger to print info
setup_logger(cfg["save_dir"], logging.INFO)

# Initialize the policy
planner = TampuraPolicy(config = cfg, problem_spec = env.problem_spec)

## Run Planner
Make sure symk is installed (see README) before running the Tampura planner.
With the default settings, the planner should pick both every time.

In [57]:
print(store)
history,store = planner.rollout(env, b0, store)

AliasStore(als={'r1': [0.0, 0.0, 0.0], 'r2': [0.0, 1.0, 0.0], 'r3': [1.0, 0.0, 0.0], 'r4': [1.0, 1.0, 0.0], 'robot_gripper': [0.5, 0.5, 0.5], 'pile': [2.0, 0.0, 0.0], 'q1': [0.0, 0.0, 0.0], 'q2': [0.0, 1.0, 0.0], 'q3': [1.0, 0.0, 0.0], 'q4': [1.0, 1.0, 0.0], 'dice': 'dice', 'o_o1': 'o_o1', 'o_o2': 'o_o2'}, als_type={'r1': 'region', 'r2': 'region', 'r3': 'region', 'r4': 'region', 'robot_gripper': 'region', 'pile': 'region', 'q1': 'robot_configuration', 'q2': 'robot_configuration', 'q3': 'robot_configuration', 'q4': 'robot_configuration', 'dice': 'physical', 'o_o1': 'physical', 'o_o2': 'physical'}, alph_counter={}, certified=[Atom(pred_name='points_to', args=['q1', 'r1']), Atom(pred_name='points_to', args=['q2', 'r2']), Atom(pred_name='points_to', args=['q3', 'r3']), Atom(pred_name='points_to', args=['q4', 'r4'])], sample_counts={}, branching_factor={})

==========t=0==========
Abstract Belief: AbstractBelief(items=[Atom(pred_name='atRob', args=['q1'])])
Reward: 0.0
Flat Stream Sampling


In [51]:
history.actions

[Action(name='look', args=['q1', 'r1']),
 Action(name='move-robot', args=['q2']),
 Action(name='look', args=['q2', 'r2']),
 Action(name='move-robot', args=['q4']),
 Action(name='look', args=['q4', 'r4']),
 Action(name='pick', args=['o_o1', 'o_gr_1', 'r1']),
 Action(name='drop', args=['o_o1', 'o_gr_1']),
 Action(name='pick', args=['o_o2', 'o_gr_2', 'r2']),
 Action(name='drop', args=['o_o2', 'o_gr_2']),
 Action(name='move-robot', args=['q3']),
 Action(name='look', args=['q3', 'r3']),
 Action(name='no-op', args=[]),
 Action(name='move-robot', args=['q1']),
 Action(name='no-op', args=[]),
 Action(name='look', args=['q1', 'r1']),
 Action(name='move-robot', args=['q2']),
 Action(name='look', args=['q2', 'r2']),
 Action(name='pick', args=['dice', 'o_gr_0', 'r2']),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(name='no-op', args=[]),
 Action(

In [6]:
history.abstract_beliefs

[AbstractBelief(items=[Atom(pred_name='at', args=['o_o1', 'r1']), Atom(pred_name='atRob', args=['q1']), Atom(pred_name='at', args=['o_o2', 'r2'])]),
 AbstractBelief(items=[Atom(pred_name='atRob', args=['q1']), Atom(pred_name='at', args=['o_o1', 'robot_gripper']), Atom(pred_name='at', args=['o_o2', 'r2']), Atom(pred_name='last_action', args=['pick'])]),
 AbstractBelief(items=[Atom(pred_name='atRob', args=['q1']), Atom(pred_name='last_action', args=['drop']), Atom(pred_name='at', args=['o_o2', 'r2']), Atom(pred_name='at', args=['o_o1', 'pile'])]),
 AbstractBelief(items=[Atom(pred_name='atRob', args=['q3']), Atom(pred_name='last_action', args=['move']), Atom(pred_name='at', args=['o_o2', 'r2']), Atom(pred_name='at', args=['o_o1', 'pile'])]),
 AbstractBelief(items=[Atom(pred_name='atRob', args=['q3']), Atom(pred_name='looked', args=['q3']), Atom(pred_name='at', args=['o_o2', 'r2']), Atom(pred_name='last_action', args=['look']), Atom(pred_name='at', args=['o_o1', 'pile'])]),
 AbstractBelief

In [7]:
store.certified

[Atom(pred_name='points_to', args=['q1', 'r1']),
 Atom(pred_name='points_to', args=['q2', 'r2']),
 Atom(pred_name='points_to', args=['q3', 'r3']),
 Atom(pred_name='points_to', args=['q4', 'r4']),
 Atom(pred_name='grasped', args=['dice', 'o_gr_0']),
 Atom(pred_name='grasped', args=['o_o1', 'o_gr_1']),
 Atom(pred_name='grasped', args=['o_o2', 'o_gr_2'])]

In [8]:
store

AliasStore(als={'r1': [0.0, 0.0, 0.0], 'r2': [0.0, 1.0, 0.0], 'r3': [1.0, 0.0, 0.0], 'r4': [1.0, 1.0, 0.0], 'robot_gripper': [0.5, 0.5, 0.5], 'pile': [2.0, 0.0, 0.0], 'q1': [0.0, 0.0, 0.0], 'q2': [0.0, 1.0, 0.0], 'q3': [1.0, 0.0, 0.0], 'q4': [1.0, 1.0, 0.0], 'dice': 'dice', 'o_o1': 'o_o1', 'o_o2': 'o_o2', 'pick': 'pick', 'drop': 'drop', 'move': 'move', 'look': 'look', 'pick-and-drop': 'pick-and-drop', 'o_gr_0': 1.0, 'o_gr_1': 1.0, 'o_gr_2': 1.0}, als_type={'r1': 'region', 'r2': 'region', 'r3': 'region', 'r4': 'region', 'robot_gripper': 'region', 'pile': 'region', 'q1': 'robot_configuration', 'q2': 'robot_configuration', 'q3': 'robot_configuration', 'q4': 'robot_configuration', 'dice': 'physical', 'o_o1': 'physical', 'o_o2': 'physical', 'pick': 'action', 'drop': 'action', 'move': 'action', 'look': 'action', 'pick-and-drop': 'action', 'o_gr_0': 'grasp', 'o_gr_1': 'grasp', 'o_gr_2': 'grasp'}, alph_counter={'gr': 3}, certified=[Atom(pred_name='points_to', args=['q1', 'r1']), Atom(pred_name